In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

import os
import sys
import glob
from collections import namedtuple

sys.path.insert(1, '/home/cem/Documents/implicit_scene/LLFF')
sys.path.insert(1, '/home/cem/Documents/implicit_scene/nerf_pl')
sys.path.insert(1, '/home/cem/Desktop/ov-workspace/src')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from imps.colmap import utils as colmap_utils
from imps.colmap.run_colmap import run_automoatic_reconstructor
from imps.nerf.data_utils import get_poses_bounds, generate_data
from imps.nerf.model import get_embedder, get_single_nerf
from imps.nerf.ray_utils import sample_points_on_ray
from imps.nerf.render import render_hierarchical, render_single_model
from imps.nerf.args import parser

# Replicate the experiment arguments
args = parser.parse_args(["--exp_name", "no_fine",
                          "--data_dir", "/home/cem/Desktop/ov-workspace/data/ov-v3",
                          "--batch_size", "1024",
                          "--down_scale", "4",
                          "--colmap"])
LOG_FOLDER = '20210921-212831'
embedding_include_input = True
log_dir = os.path.join('../logs', args.exp_name, LOG_FOLDER)

In [ ]:
# Create models

add_input = int(embedding_include_input) * 3

pos_embedder = get_embedder(args.Lp, include_input=embedding_include_input)
dir_embedder = get_embedder(args.Ld, include_input=embedding_include_input)

tf.keras.backend.clear_session()
coarse_nerf = tf.keras.models.load_model(os.path.join(log_dir, 'coarse'))
fine_nerf = tf.keras.models.load_model(os.path.join(log_dir, 'fine'))

In [ ]:
# Create data
poses_bounds = get_poses_bounds(args.data_dir, args.colmap, args.near, args.far, args.img_format, 
                                args.img_prefix)

image_files = sorted(glob.glob(os.path.join(args.data_dir, f'images/*{args.img_format}')))
origins, viewdirs, imgs, bounds = generate_data(image_files, poses_bounds, down_scale=args.down_scale)

In [ ]:
# Get first image chunk for test purposes
h_start, h_end = 40, 80
v_start, v_end = 50, 100

origins_chunk = origins[5][h_start:h_end, v_start:v_end, :]
viewdirs_chunk = viewdirs[5][h_start:h_end, v_start:v_end, :]
imgs_chunk = imgs[5][h_start:h_end, v_start:v_end, :]
bounds_chunk = bounds[5][h_start:h_end, v_start:v_end, :]

original_shape = imgs_chunk.shape
origins_chunk = tf.cast(origins_chunk.reshape(-1, 3), tf.float32)
viewdirs_chunk = tf.cast(viewdirs_chunk.reshape(-1, 3), tf.float32)
imgs_chunk = tf.cast(imgs_chunk.reshape(-1, 3), tf.float32)
bounds_chunk = tf.cast(bounds_chunk.reshape(-1, 2), tf.float32)

In [ ]:
# plt.imshow(imgs[0])
plt.imshow(imgs_chunk.numpy().reshape(*original_shape), aspect='auto')

In [ ]:
coarse_locs = sample_points_on_ray(bounds_chunk, args.N_coarse, perturb=False)
coarse_pixel, coarse_density, coarse_weights, all_pixel, all_density, all_weights, all_locs = render_hierarchical(coarse_locs, coarse_nerf, fine_nerf, args.N_fine, pos_embedder,
                                           dir_embedder, origins_chunk, viewdirs_chunk)

In [ ]:
plt.imshow(all_pixel.numpy().reshape(*original_shape), aspect='auto')

In [ ]:
depth = tf.reduce_sum(all_weights*all_density, axis=1).numpy()

In [ ]:
plt.imshow(depth.reshape(*original_shape[:2]), aspect='auto', cmap='gray')

In [ ]:
eval_image = imgs[0]
origins_eval = origins[0].reshape(-1, 3)
viewdirs_eval = viewdirs[0].reshape(-1, 3)
pixels_eval = imgs[0].reshape(-1, 3)
bounds_eval = bounds[0].reshape(-1, 2)

pred_image = np.zeros_like(pixels_eval)
pred_depth = np.zeros((pixels_eval.shape[0], 1))
test_batch_size = 2048

for b in range(0, len(bounds_eval), test_batch_size):
    origins_batch = tf.cast(origins_eval[b:b+test_batch_size], tf.float32)
    directions_batch = tf.cast(viewdirs_eval[b:b+test_batch_size], tf.float32)
    pixels_batch = tf.cast(pixels_eval[b:b+test_batch_size], tf.float32)
    bounds_batch = tf.cast(bounds_eval[b:b+test_batch_size], tf.float32)

    # Sample and perturb ray locations
    coarse_locs = sample_points_on_ray(bounds_batch, args.N_coarse, perturb=False)
    
    _, _, _, fine_rgb, fine_density, fine_weights, _  = render_hierarchical(coarse_locs, coarse_nerf, 
                                                                            fine_nerf, args.N_fine,
                                                                            pos_embedder, dir_embedder, 
                                                                            origins_batch, directions_batch)
    pred_image[b:b+test_batch_size] = fine_rgb
    pred_depth[b:b+test_batch_size] = tf.reduce_sum(fine_weights*fine_density, axis=1)

In [ ]:
plt.imshow(pred_image.reshape(*imgs[0].shape))

In [ ]:
plt.imshow(pred_depth.reshape(*imgs[0].shape[:2]), cmap='gray')